<a href="https://colab.research.google.com/github/rosembergennes/python/blob/master/Obten%C3%A7%C3%A3o_dos_Dados_B3_e_Fundamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparação do Ambiente

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install dnspython

import time
from datetime import datetime, timedelta
from selenium import webdriver
import requests
import pandas as pd
from google.colab import files
import os
import numpy as np
from pandas import DataFrame
from datetime import timedelta, date ,datetime
from decimal import Decimal
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import datetime
from pymongo import MongoClient
import pprint
import datetime
import json


### Obtenção dos Dados B3

In [3]:
# ORIGEM http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/

NOMARQ = slice(2, 15)
CODORI = slice(15, 23)
DATGER = slice(23, 31)
FILL00 = slice(31, 245)

TOTREG = slice(31, 42)
FILL99 = slice(42, 245)

# REGISTRO - 01  - COTAÇÕES HISTÓRICAS POR PAPEL-MERCADO
TIPREG = slice(0, 2)     # FIXO "01"              N(01) 
DATPRG = slice(2, 10)    # FORMATO “AAAAMMDD”     N(08)       
CODBDI = slice(10, 12)   # VER TABELA ANEXA       X(02)       
CODNEG = slice(12, 24)   #                        X(12)       
TPMERC = slice(24, 27)   # VER TABELA ANEXA       N(03)      
NOMRES = slice(27, 39)   #                        X(12) 
ESPECI = slice(39, 49)   # VER TABELA ANEXA       X(10)       
PRAZOT = slice(49, 52)   #                        X(03)       
MODREF = slice(52, 56)   #                        X(04)
PREABE = slice(56, 69)   #                        N(11)V99    
PREMAX = slice(69, 82)   #                        N(11)V99    
PREMIN = slice(82, 95)   #                        N(11)V99    
PREMED = slice(95, 108)  #                        N(11)V99    
PREULT = slice(108, 121) #                        N(11)V99   
PREOFC = slice(121, 134) #                        N(11)V99   
PREOFV = slice(134, 147) #                        N(11)V99   
TOTNEG = slice(147, 152) #                        N(05)      
QUATOT = slice(152, 170) #                        N(18)      
VOLTOT = slice(170, 188) #                        N(16)V99   
PREEXE = slice(188, 201) #                        N(11)V99   
INDOPC = slice(201, 202) # VER TABELA ANEXA       N(01)      
DATVEN = slice(202, 210) # FORMATO “AAAAMMDD”     N(08)      
FATCOT = slice(210, 217) #                        N(07)        ‘1’ = COTAÇÃO UNIT ‘1000’ = COTAÇÃO P/ LOTE DE MIL AÇÕES
PTOEXE = slice(217, 230) #                        N(07)V06     1 PONTO = 1/100 US$
CODISI = slice(230, 242) #                        X(12)      
DISMES = slice(242, 245) #                        9(03)  

tipo00 = [TIPREG, NOMARQ, CODORI, DATGER, FILL00]
nmtp00 = ['TIPREG', 'NOMARQ', 'CODORI', 'DATGER', 'FILL00']
widt00 = [2, 13, 8, 8, 214]

tipo99 = [TIPREG, NOMARQ, CODORI, DATGER, TOTREG, FILL99]
nmtp99 = ['TIPREG', 'NOMARQ', 'CODORI', 'DATGER', 'TOTREG', 'FILL99']
widt99 = [2, 13, 8, 8, 214]

tipo01 = [TIPREG, DATPRG, CODBDI, CODNEG, TPMERC, NOMRES, ESPECI, PRAZOT, MODREF, PREABE, 
          PREMAX, PREMIN, PREMED, PREULT, PREOFC, PREOFV, TOTNEG, QUATOT, VOLTOT, PREEXE, 
          INDOPC, DATVEN, FATCOT, PTOEXE, CODISI, DISMES]
nmtp01 = ['TIPREG', 'DATPRG', 'CODBDI', 'CODNEG', 'TPMERC', 'NOMRES', 'ESPECI', 'PRAZOT', 
          'MODREF', 'PREABE', 'PREMAX', 'PREMIN', 'PREMED', 'PREULT', 'PREOFC', 'PREOFV', 
          'TOTNEG', 'QUATOT', 'VOLTOT', 'PREEXE', 'INDOPC', 'DATVEN', 'FATCOT', 'PTOEXE', 
          'CODISI', 'DISMES']
widt01 = [2, 8, 2, 12, 3, 12, 10, 3, 4, 13, 13, 13, 13, 13, 13, 13, 5, 18, 18, 13, 1, 8, 7, 13, 12, 3]


In [4]:
## Leitura do arquivo pelo sistema para subir ao colab 

uploaded = files.upload()

Saving COTAHIST_D06082020.TXT to COTAHIST_D06082020.TXT


In [5]:
## importar o arquivo do colab para criação das bases

filename = os.path.join('COTAHIST_D06082020.TXT')
df_bovespa = pd.read_fwf(filename, widths=widt01, names=nmtp01, header=None, skiprows=[0], skipfooter=1)
df_bovespa.shape

(5354, 26)

### Tratamento dos Dados

In [6]:
## TRANSFORMANDO O CAMPOO DATPRG DE INT64 PARA DATETIME64  

df_bovespa["DATPRG"] = pd.to_datetime(df_bovespa['DATPRG'].astype(str), format='%Y%m%d')


In [7]:
## Removendo colunas desnecessárias para criação da tabela principal

df_new_bovespa= df_bovespa.drop(columns=[ 'TIPREG', 'TPMERC', 'PRAZOT', 'MODREF', 'PREMED', 'TOTNEG', 'QUATOT', 'PREEXE', 'INDOPC', 'DATVEN', 'FATCOT', 'PTOEXE', 'CODISI', 'DISMES']) 


In [8]:
## Transformando os valores de texto em valores separados por virgula da tabela principal

df_new_bovespa["PREABE"] = df_new_bovespa["PREABE"].apply(lambda x: int(x*10)/1000)
df_new_bovespa["PREMAX"] = df_new_bovespa["PREMAX"].apply(lambda x: int(x*10)/1000)
df_new_bovespa["PREMIN"] = df_new_bovespa["PREMIN"].apply(lambda x: int(x*10)/1000)
df_new_bovespa["PREULT"] = df_new_bovespa["PREULT"].apply(lambda x: int(x*10)/1000)
df_new_bovespa["PREOFC"] = df_new_bovespa["PREOFC"].apply(lambda x: int(x*10)/1000)
df_new_bovespa["PREOFV"] = df_new_bovespa["PREOFV"].apply(lambda x: int(x*10)/1000)
df_new_bovespa["VOLTOT"] = df_new_bovespa["VOLTOT"].apply(lambda x: int(x*10)/1000)

In [9]:
## Criação de colunas com valores de variação e percentual

df_new_bovespa['VALAF'] = df_new_bovespa['PREULT']-df_new_bovespa['PREABE']
df_new_bovespa['VALMM'] = df_new_bovespa['PREMAX']-df_new_bovespa['PREMIN']
df_new_bovespa['PERCAF'] = round((df_new_bovespa['VALAF']/df_new_bovespa['PREULT'])*100,2)
df_new_bovespa['PERCMM'] = round((df_new_bovespa['VALMM']/df_new_bovespa['PREMAX'])*100,2)

In [10]:
## Criando a tabela principal com empresas de codigo 02 e 08 das ações na base principal

df_saida_teste1 = df_new_bovespa.query('CODBDI	 == "02" ')
df_saida_teste2 = df_new_bovespa.query('CODBDI	 == "08" ')
df_saida_teste = pd.concat([df_saida_teste1, df_saida_teste2])

## Removendo empresas desnecessárias na tabela 

df_remove1 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "DRN")]
df_remove2 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "DRN ED")]
df_remove3 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "DRN C")]
df_remove4 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "DRN A")]
df_remove5 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "DR3")]
df_remove6 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "CI *")]
df_remove7 = df_saida_teste.loc[(df_saida_teste['ESPECI'] == "DR2 ATZ")]
df_remove = pd.concat([df_remove1, df_remove2, df_remove3, df_remove4, df_remove5, df_remove6, df_remove7])


df_ultimo = df_saida_teste.drop(df_remove.index)

print (df_ultimo)


        DATPRG  CODBDI CODNEG       NOMRES  ... VALAF  VALMM  PERCAF  PERCMM
11  2020-08-06       2  AALR3       ALLIAR  ...  0.67   0.75    5.90    6.60
14  2020-08-06       2  ABCB4   ABC BRASIL  ...  0.22   0.38    1.55    2.66
15  2020-08-06       2  ABEV3    AMBEV S/A  ...  0.03   0.38    0.23    2.84
19  2020-08-06       2  ADHM3  ADVANCED-DH  ... -0.06   0.07   -2.63    2.99
20  2020-08-06       2  AFLT3   AFLUENTE T  ...  0.51   0.67    4.66    6.04
..         ...     ...    ...          ...  ...   ...    ...     ...     ...
528 2020-08-06       8  TCNO3    TECNOSOLO  ... -0.05   0.06   -2.37    2.78
529 2020-08-06       8  TCNO4    TECNOSOLO  ...  0.01   0.04    0.71    2.82
530 2020-08-06       8  TEKA3         TEKA  ... -0.99   1.00   -3.67    3.57
531 2020-08-06       8  TEKA4         TEKA  ...  0.05   0.29    0.50    2.84
532 2020-08-06       8  VIVR3        VIVER  ... -0.05   0.07   -3.29    4.46

[366 rows x 16 columns]


In [11]:
df_ultimo.count

<bound method DataFrame.count of         DATPRG  CODBDI CODNEG       NOMRES  ... VALAF  VALMM  PERCAF  PERCMM
11  2020-08-06       2  AALR3       ALLIAR  ...  0.67   0.75    5.90    6.60
14  2020-08-06       2  ABCB4   ABC BRASIL  ...  0.22   0.38    1.55    2.66
15  2020-08-06       2  ABEV3    AMBEV S/A  ...  0.03   0.38    0.23    2.84
19  2020-08-06       2  ADHM3  ADVANCED-DH  ... -0.06   0.07   -2.63    2.99
20  2020-08-06       2  AFLT3   AFLUENTE T  ...  0.51   0.67    4.66    6.04
..         ...     ...    ...          ...  ...   ...    ...     ...     ...
528 2020-08-06       8  TCNO3    TECNOSOLO  ... -0.05   0.06   -2.37    2.78
529 2020-08-06       8  TCNO4    TECNOSOLO  ...  0.01   0.04    0.71    2.82
530 2020-08-06       8  TEKA3         TEKA  ... -0.99   1.00   -3.67    3.57
531 2020-08-06       8  TEKA4         TEKA  ...  0.05   0.29    0.50    2.84
532 2020-08-06       8  VIVR3        VIVER  ... -0.05   0.07   -3.29    4.46

[366 rows x 16 columns]>

In [12]:
# Ultimas Verificações
# df_ultimo.head(12)
# rascunho
# df_ultimo.loc[((df_ultimo.DATPRG == '2020-08-03')	& (df_ultimo.CODNEG == 'ABEV3')),'VPA']=88
# df_ultimo.query('CODNEG	 == "ABEV3" ')
# df_ultimo.count()
# df_ultimo.info()
df_ultimo.head()

,DATPRG,CODBDI,CODNEG,NOMRES,ESPECI,PREABE,PREMAX,PREMIN,PREULT,PREOFC,PREOFV,VOLTOT,VALAF,VALMM,PERCAF,PERCMM
11,2020-08-06,2,AALR3,ALLIAR,ON NM,10.68,11.37,10.62,11.35,11.35,11.36,9374179.0,0.67,0.75,5.90,6.60
14,2020-08-06,2,ABCB4,ABC BRASIL,PN N2,13.94,14.30,13.92,14.16,14.16,14.17,10969394.0,0.22,0.38,1.55,2.66
15,2020-08-06,2,ABEV3,AMBEV S/A,ON,13.23,13.39,13.01,13.26,13.25,13.26,462112124.0,0.03,0.38,0.23,2.84
19,2020-08-06,2,ADHM3,ADVANCED-DH,ON,2.34,2.34,2.27,2.28,2.28,2.30,10757.0,-0.06,0.07,-2.63,2.99
20,2020-08-06,2,AFLT3,AFLUENTE T,ON ED,10.43,11.10,10.43,10.94,10.64,11.00,50355.0,0.51,0.67,4.66,6.04


### WEBSCRAP

In [13]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

df_webscrap = df_ultimo #.head(7)
for index, row in df_webscrap.iterrows():

    var_CODNEG = row['CODNEG']
    # FALTA FAZER O LAÇO
    url  = 'https://www.fundamentus.com.br/detalhes.php?papel='+var_CODNEG
    print(url)
    try:
        # print("URL EXISTE NÃO DEU ERRO")
        driver.get(url)
        driver.implicitly_wait(20)      
        empresa      = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[1]/tbody/tr[3]/td[2]').text
        setor        = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[1]/tbody/tr[4]/td[2]/span/a').text
        subsetor     = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[1]/tbody/tr[5]/td[2]/span/a').text
         # cotacao   = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[1]/tbody/tr[1]/td[4]').text
        data_ult_cot = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[1]/tbody/tr[2]/td[4]/span').text 
        valmerc      = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[2]/tbody/tr[1]/td[2]/span').text
        valfirma     = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[2]/tbody/tr[2]/td[2]/span').text
        nuacoes      = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[2]/tbody/tr[2]/td[4]/span').text
        pl           = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[3]/tbody/tr[2]/td[4]/span').text 
        cresult5     = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[3]/tbody/tr[12]/td[4]/span').text
        vpa          = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[6]').text
        giroativo    = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table[3]/tbody/tr[12]/td[6]/span').text
        
        
        # trantando data de dia/mes/ano para ano-mes-dia   
        data_ult_cot = data_ult_cot[6:10]+'-'+data_ult_cot[3:5]+'-'+data_ult_cot[0:2]
             
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'EMPRESA']=empresa
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'SETOR']=setor
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'SUBSETOR']=subsetor
       # df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'COTACAO']=cotacao
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'VALMERC']=valmerc
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'VALFIRMA']=valfirma
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'NUACOES']=nuacoes
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'PL']=pl
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'CRESULT5']=cresult5
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'VPA']=vpa
        df_webscrap.loc[((df_webscrap.DATPRG == data_ult_cot)	& (df_webscrap.CODNEG == var_CODNEG)),'GIROATIVO']=giroativo

        time.sleep(10) 
        '''       
        print("VPA : " + vpa)
        print("EMPRESA : " + empresa)
        print("SETOR : " + setor)
        print("SUBSETOR : " + subsetor)
        print("COTACAO : " + cotacao)
        print("DATA ULTIMA COTACAO : " + data_ult_cot)
        '''
    except Exception as erro:
        print('Erro : ' , erro)
        vpa=0
    print('Bora Continuar')


driver.close()

https://www.fundamentus.com.br/detalhes.php?papel=AALR3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ABCB4
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ABEV3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ADHM3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=AFLT3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=AGRO3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALPA3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALPA4
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALPK3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALSO3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALUP3
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALUP4
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=ALUP11
Bora Continuar
https://www.fundamentus.com.br/detalhes.php?papel=AMAR3
Bora Continuar
https

In [14]:
# Verificando scrap dos dados
# df_webscrap.query('CODNEG	 == "ABEV3" ')
# df_webscrap.info()
# df_webscrap.head(52)
df_webscrap.head(7)

,DATPRG,CODBDI,CODNEG,NOMRES,ESPECI,PREABE,PREMAX,PREMIN,PREULT,PREOFC,PREOFV,VOLTOT,VALAF,VALMM,PERCAF,PERCMM,EMPRESA,SETOR,SUBSETOR,VALMERC,VALFIRMA,NUACOES,PL,CRESULT5,VPA,GIROATIVO
11,2020-08-06,2,AALR3,ALLIAR,ON NM,10.68,11.37,10.62,11.35,11.35,11.36,9374179.0,0.67,0.75,5.90,6.60,ALLIAR ON NM,Saúde,"Serviços Médico - Hospitalares, Análises e Dia...",1.342.630.000,1.910.140.000,118.293.000,"137,93","0,9%","10,68","0,40"
14,2020-08-06,2,ABCB4,ABC BRASIL,PN N2,13.94,14.30,13.92,14.16,14.16,14.17,10969394.0,0.22,0.38,1.55,2.66,ABC Brasil PN N2,Financeiros,Bancos,3.091.960.000,-,218.359.000,"6,31","-17,7%","18,53",-
15,2020-08-06,2,ABEV3,AMBEV S/A,ON,13.23,13.39,13.01,13.26,13.25,13.26,462112124.0,0.03,0.38,0.23,2.84,AMBEV S/A ON NM,Bebidas,Cervejas e Refrigerantes,208.648.000.000,197.947.000.000,15.735.100.000,"23,40","3,0%","4,70","0,43"
19,2020-08-06,2,ADHM3,ADVANCED-DH,ON,2.34,2.34,2.27,2.28,2.28,2.30,10757.0,-0.06,0.07,-2.63,2.99,VITALYZE.ME,Prods. de Uso Pessoal e de Limpeza,Produtos de Uso Pessoal,37.182.200,37.182.200,16.308.000,"-5,97","43,5%","-1,21","0,14"
20,2020-08-06,2,AFLT3,AFLUENTE T,ON ED,10.43,11.10,10.43,10.94,10.64,11.00,50355.0,0.51,0.67,4.66,6.04,AFLUENTE T ON,Energia Elétrica,Energia Elétrica,690.150.000,626.991.000,63.085.000,"29,04","10,7%","3,57","0,16"
21,2020-08-06,2,AGRO3,BRASILAGRO,ON NM,20.45,20.70,20.18,20.30,20.29,20.40,3045510.0,-0.15,0.52,-0.74,2.51,BRASILAGRO ON NM,Exploração de Imóveis,Exploração de Imóveis,1.260.720.000,1.806.570.000,62.104.300,"31,17","15,5%","17,77","0,34"
22,2020-08-06,2,ALPA3,ALPARGATAS,ON N1,29.00,29.75,28.59,29.75,29.56,29.90,120145.0,0.75,1.16,2.52,3.90,ALPARGATAS ON N1,"Tecidos, Vestuário e Calçados",Calçados,17.494.800.000,17.353.300.000,588.062.000,"65,81","-4,4%","4,85","0,48"


In [15]:
df_webscrap["VALMERC"]=df_webscrap["VALMERC"].replace(to_replace ="-",  value ="0")
df_webscrap["VALFIRMA"]=df_webscrap["VALFIRMA"].replace(to_replace ="-",  value ="0")
df_webscrap["NUACOES"]=df_webscrap["NUACOES"].replace(to_replace ="-",  value ="0")
df_webscrap["PL"]=df_webscrap["PL"].replace(to_replace ="-",  value ="0") 
df_webscrap["CRESULT5"]=df_webscrap["CRESULT5"].replace(to_replace ="-",  value ="0") 
df_webscrap["VPA"]=df_webscrap["VPA"].replace(to_replace ="-",  value ="0") 
df_webscrap["GIROATIVO"]=df_webscrap["GIROATIVO"].replace(to_replace ="-",  value ="0") 

In [16]:
# Ao obter os dados do WEBSCRAP ocorre necessidade de tratamento de alguns campos 
df_webscrap["VALMERC"] = df_webscrap["VALMERC"].str.replace('.','')
df_webscrap["VALFIRMA"] = df_webscrap["VALFIRMA"].str.replace('.','')
df_webscrap["NUACOES"] = df_webscrap["NUACOES"].str.replace('.','')

df_webscrap["PL"] = df_webscrap["PL"].str.replace('.','x')
df_webscrap["PL"] = df_webscrap["PL"].str.replace(',','.')
df_webscrap["PL"] = df_webscrap["PL"].str.replace('x','')

df_webscrap["CRESULT5"] = df_webscrap["CRESULT5"].str.replace('%','0')
df_webscrap["CRESULT5"] = df_webscrap["CRESULT5"].str.replace('.','x')
df_webscrap["CRESULT5"] = df_webscrap["CRESULT5"].str.replace(',','.')
df_webscrap["CRESULT5"] = df_webscrap["CRESULT5"].str.replace('x','')

df_webscrap["VPA"] = df_webscrap["VPA"].str.replace('.','x')
df_webscrap["VPA"] = df_webscrap["VPA"].str.replace(',','.')
df_webscrap["VPA"] = df_webscrap["VPA"].str.replace('x','')
df_webscrap["GIROATIVO"] = df_webscrap["GIROATIVO"].str.replace('.','x')
df_webscrap["GIROATIVO"] = df_webscrap["GIROATIVO"].str.replace(',','.')
df_webscrap["GIROATIVO"] = df_webscrap["GIROATIVO"].str.replace('x','')

In [17]:
# Tansformo as colunas de string para float
df_webscrap["VALMERC"] = df_webscrap["VALMERC"].astype('float64',(2,))
df_webscrap["VALFIRMA"] = df_webscrap["VALFIRMA"].astype('float64',(2,))
df_webscrap["NUACOES"] = df_webscrap["NUACOES"].astype('float64',(2,))
df_webscrap["PL"] = df_webscrap["PL"].astype('float64')
df_webscrap["CRESULT5"] = df_webscrap["CRESULT5"].astype('float64')
df_webscrap["VPA"] = df_webscrap["VPA"].astype('float64')
df_webscrap["GIROATIVO"] = df_webscrap["GIROATIVO"].astype('float64')


In [18]:
df_webscrap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 11 to 532
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DATPRG     366 non-null    datetime64[ns]
 1   CODBDI     366 non-null    int64         
 2   CODNEG     366 non-null    object        
 3   NOMRES     366 non-null    object        
 4   ESPECI     366 non-null    object        
 5   PREABE     366 non-null    float64       
 6   PREMAX     366 non-null    float64       
 7   PREMIN     366 non-null    float64       
 8   PREULT     366 non-null    float64       
 9   PREOFC     366 non-null    float64       
 10  PREOFV     366 non-null    float64       
 11  VOLTOT     366 non-null    float64       
 12  VALAF      366 non-null    float64       
 13  VALMM      366 non-null    float64       
 14  PERCAF     366 non-null    float64       
 15  PERCMM     366 non-null    float64       
 16  EMPRESA    360 non-null    object        
 

### INICIANDO GRAVAÇÃO <<< DIARIA >>>

In [19]:
client = MongoClient("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
db = client.acoes


In [20]:
# records = json.loads(df_webscrap.T.to_json()).values()
# db.bolsabrasil.insert(records)

# records = json.loads(df_webscrap.T.to_json()).values()
# db.bolsabrasil.insert(records)

db.bolsabrasil.insert_many(df_webscrap.to_dict('records'))

## Fim Armazenagem